In [1]:
#!pip install tsfresh --user
#!pip install pytorch_toolbelt --user

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
#import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import pickle
import gc
import random
import os
import time
#from apex import amp
from sklearn.model_selection import KFold
from tsfresh.feature_extraction.feature_calculators import linear_trend, kurtosis, quantile, skewness, mean_second_derivative_central, sample_entropy
from tqdm.notebook import tqdm
from pytorch_toolbelt import losses as L
from scipy import signal
os.listdir('./input')

['sample_submission.csv',
 'Y_test_proba.npy',
 'train.csv',
 'test_kalman.csv',
 'test.csv',
 'Y_train_proba.npy',
 'train_kalman.csv',
 'liverpool-ion-switching.zip']

In [3]:
print(torch.__version__) 

1.4.0


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train = pd.read_csv('./input/train_kalman.csv')
test = pd.read_csv('./input/test_kalman.csv')
test_pseudo_labels = pd.read_csv("submission_20200426.csv")['open_channels'].values
Y_train_proba = np.load("./input/Y_train_proba.npy")
Y_test_proba  = np.load("./input/Y_test_proba.npy") 

for i in range(11):
    train[f"proba_{i}"] = Y_train_proba[:, i]
    test[f"proba_{i}"] = Y_test_proba[:, i]

num_class  = len(train['open_channels'].unique())
lr         = 1e-3
batch_size = 64
multi_step = [30, 50, 60]
epochs     = 100
n_fold     = 5
seq_length = 1000

## Feature Engineering

In [ ]:
def feature_generator_1(df, window_sizes, groups, 
                        add_pct_change = False,
                        shift_sizes = None, 
                        add_signal_shift = False,
                        add_signal_shift_lag = False):
    
    no_substract_cols = []
    
    df['signal_2'] = df['signal'] ** 2
    
    if add_pct_change:
        for group_id in groups:
            no_substract_cols.append('pct_change')
            idx = np.arange(group_id*500000, (group_id+1)*500000)
            df.loc[idx, 'pct_change'] = df[df.group == group_id]['signal'].pct_change()
    
    for window in tqdm(window_sizes):
        no_substract_cols.append("rolling_std_" + str(window))
        no_substract_cols.append("rolling_max_" + str(window))
        no_substract_cols.append("rolling_mean_" + str(window))
            
        for group_id in groups:
            idx = np.arange(group_id*500000, (group_id+1)*500000)
            df.loc[idx, "rolling_mean_" + str(window)] = df[df.group == group_id]['signal'].rolling(window=window).mean()
            df.loc[idx, "rolling_std_" + str(window)]  = df[df.group == group_id]['signal'].rolling(window=window).std()
            df.loc[idx, "rolling_min_" + str(window)]  = df[df.group == group_id]['signal'].rolling(window=window).min()
            df.loc[idx, "rolling_max_" + str(window)]  = df[df.group == group_id]['signal'].rolling(window=window).max()

        
        df["rolling_min_" + str(window)+'_s'] = df["rolling_min_" + str(window)] - df.signal
        df["rolling_std_" + str(window)+'_s'] = df["rolling_std_" + str(window)] - df.signal
        df = df.drop(columns=["rolling_min_" + str(window), "rolling_std_" + str(window)])
    
    if shift_sizes is not None:
        for shift_size in tqdm(shift_sizes):
            no_substract_cols.append('pct_change_shift_pos_'+str(shift_size))
            no_substract_cols.append('pct_change_shift_neg_'+str(shift_size))
            if add_pct_change:
                df.loc[idx , 'pct_change_shift_pos_'+str(shift_size)] = df[df.group == group_id]['pct_change'].shift(shift_size).astype(np.float64)
                df.loc[idx , 'pct_change_shift_neg_'+str(shift_size)] = df[df.group == group_id]['pct_change'].shift(-1*shift_size).astype(np.float64)
            if add_signal_shift:
                df.loc[idx , 'signal_shift_pos_'+str(shift_size)] = df[df.group == group_id]['signal'].shift(shift_size).astype(np.float64)
                df.loc[idx , 'signal_shift_neg_'+str(shift_size)] = df[df.group == group_id]['signal'].shift(-1*shift_size).astype(np.float64)
            if add_signal_shift_lag:        
                df['signal_shift_pos_'+str(shift_size)+'_s'] = df['signal_shift_pos_'+str(shift_size)] - df.signal
                df['signal_shift_neg_'+str(shift_size)+'_s'] = df['signal_shift_neg_'+str(shift_size)] - df.signal
                df = df.drop(columns=['signal_shift_pos_'+str(shift_size), 'signal_shift_neg_'+str(shift_size)])   
                    
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    
    return df, no_substract_cols

def feature_generator_2(df, batch_sizes, windows_sizes=None):
    
    df = df.sort_values(by=['time']).reset_index(drop=True)
    df.index = ((df.time * 10_000) - 1).values
    
    for batch_size in tqdm(batch_sizes):
        
        df['batch']         = df.index // batch_size
        df['batch_index']   = df.index  - (df.batch * batch_size)
        df['batch_slices']  = df['batch_index']  // (batch_size / 10)
        df['batch_slices2'] = df['batch'].astype(str).str.zfill(3) + '_' + df['batch_slices'].astype(str).str.zfill(3)
        
        for c in ['batch', 'batch_slices2']:
            d = {}
            d['mean_{}_{}'.format(c, batch_size)]   = df.groupby([c])['signal'].mean()
            d['median_{}_{}'.format(c, batch_size)] = df.groupby([c])['signal'].median()
            d['max_{}_{}'.format(c, batch_size)]    = df.groupby([c])['signal'].max()
            d['min_{}_{}'.format(c, batch_size)]    = df.groupby([c])['signal'].min()
            d['std_{}_{}'.format(c, batch_size)]    = df.groupby([c])['signal'].std()
            d['mean_abs_chg_{}_{}'.format(c, batch_size)] = df.groupby([c])['signal'].apply(lambda x: np.mean(np.abs(np.diff(x))))
            d['abs_max_{}_{}'.format(c, batch_size)] = df.groupby([c])['signal'].apply(lambda x: np.max(np.abs(x)))
            d['abs_min_{}_{}'.format(c, batch_size)] = df.groupby([c])['signal'].apply(lambda x: np.min(np.abs(x)))
            for v in d:
                df[v] = df[c].map(d[v].to_dict())
            df['range_{}_{}'.format(c, batch_size)] = df['max_{}_{}'.format(c, batch_size)] - df['min_{}_{}'.format(c, batch_size)]
            df['maxtomin_{}_{}'.format(c, batch_size)] = df['max_{}_{}'.format(c, batch_size)] / df['min_{}_{}'.format(c, batch_size)]
            df['abs_avg_{}_{}'.format(c, batch_size)] = (df['abs_min_{}_{}'.format(c, batch_size)] + df['abs_max_{}_{}'.format(c, batch_size)]) / 2
            
            #substruct features
            df['min_{}_{}'.format(batch_size, c)+ '_s'] = df['min_{}_{}'.format(c, batch_size)] - df['signal']
            df['std_{}_{}'.format(batch_size, c)+ '_s'] = df['std_{}_{}'.format(c, batch_size)] - df['signal']
            df['mean_abs_chg_{}_{}'.format(batch_size, c) + '_s'] = df['mean_abs_chg_{}_{}'.format(c, batch_size)] - df['signal']
            df['abs_min_{}_{}'.format(batch_size, c)+ '_s'] = df['abs_min_{}_{}'.format(c, batch_size)] - df['signal']
            df['abs_avg_{}_{}'.format(batch_size, c) + '_s'] = df['abs_avg_{}_{}'.format(c, batch_size)] - df['signal']
            
            df = df.drop(columns=['min_{}_{}'.format(c, batch_size), 
                                  'abs_min_{}_{}'.format(c, batch_size),
                                  'abs_avg_{}_{}'.format(c, batch_size)])
    
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    
    return df

In [ ]:
def add_bathing_to_data(df : pd.DataFrame) -> pd.DataFrame :
    batches = df.shape[0] // 500000
    df['group'] = 0
    for i in range(batches):
        idx = np.arange(i*500000, (i+1)*500000)
        df.loc[idx, 'group'] = i
        
    return df, df.group.unique()

train, train_group = add_bathing_to_data(train)
test, test_group   = add_bathing_to_data(test)

In [ ]:
windows_sizes=[1000, 5000]
train, no_substract_cols = feature_generator_1(train, 
                                               windows_sizes, 
                                               train_group, 
                                               add_pct_change=False, 
                                               shift_sizes=None, 
                                               add_signal_shift=False,
                                               add_signal_shift_lag=False)


test, _ = feature_generator_1(test, 
                              windows_sizes, 
                              test_group, 
                              add_pct_change=False, 
                              shift_sizes=None, 
                              add_signal_shift=False,
                              add_signal_shift_lag=False)

In [ ]:
train = feature_generator_2(train, batch_sizes=[50000])
test  = feature_generator_2(test, batch_sizes=[50000]) 

In [ ]:
train_y = train['open_channels'].values


train_x = train.drop(columns=['time', 'open_channels', 'group',
                              'batch', 
                              'batch_index', 
                              'batch_slices', 
                              'batch_slices2',
                             ])

test.drop(columns=['time', 
                   'batch', 
                   'group',
                   'batch_index', 
                   'batch_slices', 
                   'batch_slices2',
                  ], inplace=True)

In [ ]:
train_x.columns

In [ ]:
del train
gc.collect()

In [ ]:
scaler = StandardScaler()
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x_scaled  = scaler.transform(test)

with open('scaler.pickle', mode='wb') as f:
    pickle.dump(scaler, f)

In [ ]:
train_x = np.concatenate([train_x, test_x_scaled], 0)
train_y = np.concatenate([train_y, test_pseudo_labels], 0)

In [ ]:
train_x_scaled_reshaped = train_x.reshape(-1, seq_length, train_x.shape[1])
train_y                 = train_y.reshape(-1, seq_length)
test_x_scaled_reshaped  = test_x_scaled.reshape(-1, seq_length, test_x_scaled.shape[1])
print(train_x_scaled_reshaped.shape)
print(test_x_scaled_reshaped.shape)

In [ ]:
del train_x
del test_x_scaled
gc.collect()

## DataLoader

In [ ]:
class IonDataset(Dataset):

    def __init__(self, X, y=None, augment=False):
                
        self.inputs = X
        self.labels = y
        self.augment = augment

    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self, idx):
        
        input = self.inputs[idx]
        
        if self.labels is not None:
            
            if self.augment:
                if np.random.rand() < 0.5:
                    return np.flip(input, 0).copy(), np.flip(self.labels[idx],0).copy()
                else:
                    return input, self.labels[idx]
            return input, self.labels[idx]
        else:
            return input

# Model

In [ ]:
class IonModel(nn.Module):
    
    def __init__(self, input_dim, num_class, num_hidden):
        super(IonModel, self).__init__()

        self.gru1     = nn.GRU(input_dim, num_hidden, batch_first=True, bidirectional=True)
        self.dropout  = nn.Dropout(0.4)
        self.gru2     = nn.GRU(num_hidden*2, num_hidden, batch_first=True, bidirectional=True)
        self.gru3     = nn.GRU(num_hidden*4, num_hidden, batch_first=True, bidirectional=True)
        
        self.fc = nn.Sequential(nn.Linear(num_hidden*6, num_hidden),
                                nn.ReLU(inplace=True),
                                nn.Dropout(0.1),
                                nn.Linear(num_hidden, num_hidden),
                                nn.ReLU(inplace=True),
                                nn.Linear(num_hidden, num_class))
    
        
    
    def forward(self, input):
        
        gru1, _ = self.gru1(input)
        gru1    = self.dropout(gru1)
        gru2, _ = self.gru2(gru1)
        gru2    = self.dropout(gru2)
        gru2    = torch.cat((gru1, gru2), 2)
        gru3, _ = self.gru3(gru2)
        gru3    = self.dropout(gru3)
        gru3    = torch.cat((gru2, gru3), 2)
        output  = self.fc(gru3)
        
        return output

In [ ]:
def model_test():
    m = IonModel(31, 10, 64)
    input = torch.randn(20, 16, 31)
    print(m(input).size())

In [ ]:
model_test()

## Train Model

In [ ]:
def train_model(model, criterion, train_loader, optimizer):
    max_grad_norm = 1.0
    avg_loss = 0.0
    model.train()
    for idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.float().to(device)
        labels = labels.long().to(device)
        output_train = model(inputs)
        loss = criterion(torch.transpose(output_train, 1, 2), labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()
        avg_loss += loss.item() / len(train_loader)
        
    return avg_loss

def val_model(model, criterion, val_loader):
    avg_val_loss = 0.0
    preds = []
    original = []
    model.eval()
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.float().to(device)
            labels = labels.long().to(device)
            output_val = model(inputs)
            loss = criterion(torch.transpose(output_val, 1, 2), labels)
            avg_val_loss += loss.item() / len(val_loader)
            preds.append(np.argmax(F.softmax(output_val, 2).detach().cpu().numpy(), 2).reshape(-1))
            original.append(labels.detach().cpu().numpy().reshape(-1))
        
        truths = np.concatenate(original)
        preds  = np.concatenate(preds)
        
        score = f1_score(truths, preds, average='macro')
        
    return avg_val_loss, score

In [ ]:
best_sub = pd.read_csv('submission_20200426.csv').open_channels.values.astype(int)

In [ ]:
def predict_result(model, test_loader, batch_size=16):
    
    output = np.zeros((len(test), num_class))
    model.eval()
    with torch.no_grad():
        for idx, inputs in enumerate(test_loader):
            start_index = idx * batch_size*seq_length
            end_index   = min(start_index + batch_size*seq_length, len(test))
            inputs = inputs.float().to(device)
            preds  = model(inputs)
            preds  = F.softmax(preds, 2).detach().cpu().numpy().reshape(-1, num_class)
            output[start_index:end_index, :] = preds
            
    result_test = np.argmax(output, 1).astype(np.int)
    print(f1_score(best_sub, result_test, average='macro'))
    
    return output

In [ ]:
def train_model_kfold():
    
    seed_everything(2020)
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
    
    test_set    = IonDataset(test_x_scaled_reshaped)
    test_loader = DataLoader(test_set, batch_size=16, shuffle=False)
    result      = 0
    
    for fold_n, (train_index, valid_index) in enumerate(folds.split(train_x_scaled_reshaped)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = train_x_scaled_reshaped[train_index], train_x_scaled_reshaped[valid_index]
        y_train, y_valid = train_y[train_index], train_y[valid_index]
    
        train_set    = IonDataset(X_train, y_train, augment=False)
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
        val_set      = IonDataset(X_valid, y_valid)
        val_loader   = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    
        model = IonModel(train_x_scaled_reshaped.shape[-1], num_class, 128)
        model.to(device)
        
        best_avg_loss    = 100.0
        best_score       = 0.0
        best_param_loss  = None
        best_param_score = None
        i = 0
        
        optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=1e-6)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        criterion = L.FocalLoss()
        
        for epoch in range(epochs):
        
            #if i == 10: break
            start_time          = time.time()
            avg_loss            = train_model(model, criterion, train_loader, optimizer)
            avg_val_loss, score = val_model(model, criterion, val_loader)
            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t score={:.6f} \t time={:.2f}s'.format(epoch + 1,
                                                                                                         epochs, 
                                                                                                         avg_loss, 
                                                                                                         avg_val_loss,
                                                                                                         score, 
                                                                                                         elapsed_time))
    
            if best_avg_loss > avg_val_loss:
                i = 0
                best_avg_loss = avg_val_loss 
                best_param_loss = model.state_dict()
            if best_score < score:
                best_score = score
                best_param_score = model.state_dict()
            else:
                i += 1
            scheduler.step()
            
        model.load_state_dict(best_param_score)
        result += predict_result(model, test_loader)
        
    return result

In [ ]:
result = train_model_kfold()
result /= n_fold
final_output = np.argmax(result, 1).astype(np.int)

In [ ]:
sample_df = pd.read_csv("./input/sample_submission.csv", dtype={'time':str})
sample_df['open_channels'] = final_output
sample_df.to_csv("submission_gru_only_20200427_00.csv", index=False, float_format='%.4f')

In [ ]:
sample_df.head()

In [ ]:
#0.9887122108964345
f1_score(best_sub, sample_df.open_channels, average='macro')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,5))
res = 1000
let = ['A','B','C','D','E','F','G','H','I','J']
plt.plot(range(0,test.shape[0],res),sample_df.open_channels[0::res])
plt.plot(range(0,test.shape[0],res),test.signal[0::res])
for i in range(5): plt.plot([i*500000,i*500000],[-5,12.5],'r')
for i in range(21): plt.plot([i*100000,i*100000],[-5,12.5],'r:')
for k in range(4): plt.text(k*500000+250000,10,str(k+1),size=20)
for k in range(10): plt.text(k*100000+40000,7.5,let[k],size=16)
plt.title('Test Data Predictions',size=16)
plt.show()